# Utilities to help prediction loop

In [1]:
import predict_utils as pred 

In [4]:
scores, predictions, ground_truth, alphas = pred.get_predictions("model_new", "preprocessed/try_2/", "test", model_type="baseline")


Running baseline from C:\Users\apra\Desktop\FALL 2021\CY 7990\toxicity_filter_attacks\model_new\model_noatt_checkpoints\best_model
padded sent: (2308, 100)
feature shape: (2308, 100)
padded pos sentences: (2308, 100)
debug padded_pos_sentences: ['A', ',', '^', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>']
pos feature shape: (2308, 100)
Running model on 2308 test samples
No supervised attention.
INFO:tensorflow:Restoring parameters from C:\Users\apra\Desktop\FALL 2021\CY 7990\toxicity_filter_attacks\model_new\model_noatt_checkpoints\best_model


In [5]:
pred.print_metrics(ground_truth,scores)

Done
----------------------------------------------------------------------------------------------------
Predictions:
[[0.08187434, 0.9181256], [0.09040463, 0.9095954], [0.08228504, 0.9177149], [0.23696016, 0.7630398], [0.46449992, 0.53550005], [0.80933416, 0.19066581], [0.082129866, 0.91787016], [0.088497974, 0.91150206], [0.7064501, 0.29354993], [0.17605793, 0.823942]]
Ground Truth:
[1, 0, 0, 0, 1, 1, 0, 1, 1, 1]
Classification:
[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]
METRICS: 

roc_auc: 0.8020886102668017
pr_auc: 0.5901431362169314
fscore: 0.5722937168811507
accuracy: 0.7551993067590987


In [17]:
import FLAGS
import tensorflow as tf
import os
print(tf.__version__)
from model.abuse_classifier import AbuseClassifier
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from data_utils_v2.data_helpers import genFeatures, loadVocabEmb,genPOSFeatures, loadData
import param
import pickle

1.15.0


In [26]:
BEST_THRESHOLD = 0.3

In [27]:
def get_predictions(model_folder_path, data_folder_path, data_type="test", dump_folder_path = "dump_2",  model_type="baseline"):
    with open(os.path.join(dump_folder_path, "norm_init_embed.pkl"), "rb") as handle:
        init_embed = pickle.load(handle)

    
    #configure model type
    if model_type == "baseline":
        attention_lambda = 0.0
        attention_loss_type = "none"
        model_type_path = "model_noatt_checkpoints"
    else:
        attention_lambda = 0.2
        attention_loss_type = "encoded"
        model_type_path = "model_att=encoded_checkpoints"
        
    checkpoint_dir = os.path.abspath(os.path.join(model_folder_path, model_type_path))
    model_path = os.path.join(checkpoint_dir, "best_model")
    print(f"Running {model_type} from {model_path}")
    
    
    
    x_test, length_test, attention_test, pos_test, pos_length_test, y_test = loadData(dump_folder_path, data_folder_path, data_type,  verbose=False)
    
    len_data = len(x_test)
    print(f"Running model on {len_data} {data_type} samples")
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
            allow_soft_placement=FLAGS.allow_soft_placement,
            log_device_placement=FLAGS.log_device_placement
        )
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            model = AbuseClassifier(
                max_sequence_length=param.max_sent_len,
                num_classes=2,
                pos_vocab_size=FLAGS.pos_vocab_size,
                init_embed=init_embed,
                hidden_size=FLAGS.hidden_size,
                attention_size=FLAGS.attention_size,
                keep_prob=FLAGS.dropout_keep_prob,
                attention_lambda=attention_lambda,
                attention_loss_type=attention_loss_type,
                l2_reg_lambda=0.1,
                use_pos_flag=FLAGS.use_pos_flag)

            global_step = tf.Variable(0, name="global_step", trainable=False)
            saver = tf.train.Saver(tf.all_variables())
            # Initialize all variables
            sess.run(tf.initialize_all_variables())
            saver.restore(sess, model_path)

            dev_scores = []
            dev_confidences = []
            alphas = []
            pos = 0
            gap = 50
            while pos < len(x_test):
                x_batch = x_test[pos:pos + gap]
                pos_batch = pos_test[pos:pos + gap]
                y_batch = y_test[pos:pos + gap]
                length_batch = length_test[pos:pos + gap]
                pos_length_batch = pos_length_test[pos:pos + gap]
                pos += gap
                # score sentences
                feed_dict = {
                    model.input_word: x_batch,
                    model.input_pos: pos_batch,
                    model.input_y: y_batch,
                    model.sequence_length: length_batch,
                    model.dropout_keep_prob: 1.0
                }
                step, scores, alpha = sess.run([global_step, model.prob, model.alphas], feed_dict)
                dev_confidences = dev_confidences + list([[s[0],s[1]] for s in scores])
                dev_scores = dev_scores + list([s[0] for s in scores])
                alphas = alphas + list(alpha)
    ground_truth = [g[0] for g in y_test]
    return dev_confidences, dev_scores, ground_truth, alphas




In [30]:
scores, predictions, ground_truth, alphas = get_predictions("model_new", "preprocessed/try_2/", "test", model_type="attention")



print("Done")
print("-" * 100)


print("Predictions:")
print(scores[0:10])

print("Ground Truth:")
print(ground_truth[0:10])

print("Classification:")
y_pred = list(map(lambda p: 1 if p > BEST_THRESHOLD else 0, predictions))
print(y_pred[0:10])

Running attention from C:\Users\apra\Desktop\FALL 2021\CY 7990\toxicity_filter_attacks\model_new\model_att=encoded_checkpoints\best_model
padded sent: (2308, 100)
feature shape: (2308, 100)
padded pos sentences: (2308, 100)
debug padded_pos_sentences: ['A', ',', '^', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>']
pos feature shape: (2308, 100)
Running model on 2308 test samples
Supervised attention with encoded loss.
INFO:tensorflow:Restoring parameters from C:\Users\apra\Desktop\FALL 2021\CY 7990\toxicity_filter_attacks\model_new\model_att=encoded_checkpoints\best_model
Done
----------------------------------------------------------------------------------------------------
Predictions:
[[0.32269272, 0.6773073], [0.34807494, 0.6519251], [0.09782139, 0.9021786], [0.7724647, 0.22753535], [0.8217768, 0.17822321], [0.85405266, 0.14594728], [0.09517542, 0.9048246], [0.162968, 0.83703196], [0.7225079, 0.27749208], [0.09746935, 0.9025306]]
Ground Truth:
[1, 0, 0, 0, 1, 1, 0, 

In [31]:
from data_utils import eval_helpers
from sklearn.metrics import f1_score, accuracy_score
print("TEST METRICS: \n")
# roc auc
eval_helpers.evalROC(ground_truth, predictions)
# pr auc
eval_helpers.evalPR(ground_truth, predictions)
# f1 score
fscore = f1_score(ground_truth, y_pred)
print("fscore: {}".format(fscore))

accuracy = accuracy_score(ground_truth,y_pred)
print("accuracy: {}".format(fscore))

TEST METRICS: 

roc_auc: 0.792010626784563
pr_auc: 0.6228769778862439
fscore: 0.5803108808290155
accuracy: 0.5803108808290155
